In [1]:
# Data visualization
import matplotlib.pyplot as plt 

# Data manipulation
import pandas as pd
import numpy as np
import csv
from zipfile import ZipFile

In [2]:
# Webscraping
import glob
import requests
from bs4 import BeautifulSoup
import time
import datetime
from pandas.core.common import flatten
import os
from itertools import chain
from tqdm import tqdm
import json
import urllib.request

In [3]:
# Parsing and pre-processing
from glob import glob
import os 
import re

from pdfminer.high_level import extract_text
import pdfplumber
from langdetect import detect, DetectorFactory

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from string import punctuation
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [4]:
import sys
sys.path.append(f'../../python')
from parser import parse_pdf, parse_pdf_section, clean_parsed_pdf_section

In [5]:
pdf_dir = f"../../../data/pdfs/"
art8_1_pdf_list = glob(os.path.join(pdf_dir, "art8.1/*/*.pdf")) #36
art8_1_pdf_list.sort(key=lambda x: os.path.getctime(x))

art8_2_pdf_list = glob(os.path.join(pdf_dir, "art8.2/*/*.pdf")) #72
art8_2_pdf_list.sort(key=lambda x: os.path.getctime(x))

art8_3_pdf_list = glob(os.path.join(pdf_dir, "art8.3/*/*.pdf")) #14
art8_3_pdf_list.sort(key=lambda x: os.path.getctime(x))

art6_1_pdf_list = glob(os.path.join(pdf_dir, "art6.1/*/*.pdf"))
art6_1_pdf_list.sort(key=lambda x: os.path.getctime(x))

art6_0_pdf_list = glob(os.path.join(pdf_dir, "art6.0/*/*.pdf"))
art6_0_pdf_list.sort(key=lambda x: os.path.getctime(x))

art9_3_pdf_list = glob(os.path.join(pdf_dir, "art9.3/*/*.pdf"))
art9_3_pdf_list.sort(key=lambda x: os.path.getctime(x))


### Fix parser by section (try)

In [6]:
df = parse_pdf_section(art8_1_pdf_list[0:2])

100%|██████████| 2/2 [00:09<00:00,  4.56s/it]


In [7]:
df_clean = clean_parsed_pdf_section(df)

In [8]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37 entries, 0 to 36
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          37 non-null     object
 1   year          37 non-null     object
 2   len_pdf       37 non-null     object
 3   article       37 non-null     object
 4   article_txt   37 non-null     object
 5   article_62    37 non-null     object
 6   case_num      37 non-null     object
 7   filename      37 non-null     object
 8   section_text  37 non-null     object
 9   bsn_act       37 non-null     object
 10  simp_text     37 non-null     object
 11  section       37 non-null     object
dtypes: object(12)
memory usage: 3.6+ KB


In [9]:
df_60 = parse_pdf_section(art6_0_pdf_list[0:2])

100%|██████████| 2/2 [00:00<00:00,  3.93it/s]


In [10]:
df_60_clean = clean_parsed_pdf_section(df_60)

In [12]:
df_60_clean

,date,year,len_pdf,article,article_txt,article_62,case_num,filename,section_text,bsn_act,simp_text,section
0,Date: 01/12/2020,2020,3,art6.0,article6(1)(b),None,M.10011,art6.0\M.10011\m10011_48_3,None,EUROPEAN COMMISSION \nDG Competition \n \n \n ...,: \n — ORIX is a multinational integrated fi...,NaN
1,Date: 17/11/2020,2020,3,art6.0,article6(1)(b),None,M.10000,art6.0\M.10000\m10000_41_3,None,\n \nEUROPEAN COMMISSION \nDG Competition \n ...,: \n PreZero provides waste disposal and rec...,NaN


In [ ]:
#TODO add code to those that do not detect subheadings right

#### Article 8.1 (36 cases)

In [13]:
df_81=parse_pdf_section(art8_1_pdf_list)

100%|██████████| 47/47 [08:59<00:00, 11.49s/it]


In [14]:
df_81_clean = clean_parsed_pdf_section(df_81)

In [15]:
# save json file name
date = datetime.date.today().strftime('%Y_%m_%d')

file_name = f"../../../data/parsed/parsed_art8.1_{date}.json"
if os.path.exists(file_name):
    os.remove(file_name)

# save file as json
df_81_clean.to_json(file_name)

#### Article 8.2 (73 cases)

In [8]:
# create dataframe
df_82 = pd.DataFrame(art8_2_pdf_list, columns = ['pdf_file'])

df_82['id'] = df_82.index
df_82['batch'] = ((df_82['id'])/20).astype(int)

In [9]:
df_82['batch'].value_counts()

0    20
1    20
2    20
3    20
4    20
5    14
Name: batch, dtype: int64

In [10]:
# parse pdf
for i in range(0,6): # 0-5
    globals()[f'df_8.2_{i}'] = clean_parsed_pdf_section(parse_pdf_section(tqdm(df_82[df_82['batch']==i]['pdf_file'])))
    print("Batch {} completed".format(i))

    # save json file name
    date = datetime.date.today().strftime('%Y_%m_%d')

    file_name = f"../../../data/parsed/parsed_art8.2_batch{i}_{date}.json"
    if os.path.exists(file_name):
        os.remove(file_name)

    # save file as json
    globals()[f'df_8.2_{i}'].to_json(file_name)

100%|██████████| 20/20 [07:55<00:00, 23.79s/it]


Batch 0 completed


100%|██████████| 20/20 [11:55<00:00, 35.77s/it]


Batch 1 completed


100%|██████████| 20/20 [12:20<00:00, 37.04s/it]


...

Batch 3 completed


100%|██████████| 20/20 [04:29<00:00, 13.45s/it]


Batch 4 completed


100%|██████████| 14/14 [5:02:04<00:00, 1294.59s/it]


Batch 5 completed


#### Article 8.3 (13 cases)

In [6]:
df_83 = parse_pdf_section(art8_3_pdf_list)

100%|██████████| 14/14 [11:28<00:00, 49.21s/it]


In [ ]:
df_83_clean = clean_parsed_pdf_section(df_83)

In [13]:
# save json file name
date = datetime.date.today().strftime('%Y_%m_%d')

file_name = f"../../../data/parsed/parsed_art8.3_{date}.json"
if os.path.exists(file_name):
    os.remove(file_name)

# save file as json
df_83_clean.to_json(file_name)

#### Article 6.1(b)_cc (240 cases)

In [6]:
# create dataframe
df_61 = pd.DataFrame(art6_1_pdf_list, columns = ['pdf_file'])

df_61['id'] = df_61.index
df_61['batch'] = ((df_61['id'])/100).astype(int)

In [12]:
df_61['batch'].value_counts()

0    100
1    100
2    100
3     59
Name: batch, dtype: int64

In [7]:
# parse pdf
for i in range(0,4): # 0-3
    globals()[f'df_6.1_{i}'] = clean_parsed_pdf_section(parse_pdf_section(tqdm(df_61[df_61['batch']==i]['pdf_file'])))
    print("Batch {} completed".format(i))

    # save json file name
    date = datetime.date.today().strftime('%Y_%m_%d')

    file_name = f"../../../data/parsed/parsed_art6.1_batch{i}_{date}.json"
    if os.path.exists(file_name):
        os.remove(file_name)

    # save file as json
    globals()[f'df_6.1_{i}'].to_json(file_name)

100%|██████████| 100/100 [10:03<00:00,  6.03s/it]


Batch 0 completed


100%|██████████| 100/100 [13:46<00:00,  8.27s/it]


Batch 1 completed


100%|██████████| 100/100 [12:28<00:00,  7.48s/it]


Batch 2 completed


100%|██████████| 59/59 [06:51<00:00,  6.97s/it]

Batch 3 completed


#### Article 6(b)_uc (5734 cases)

In [6]:
# create dataframe
df_60 = pd.DataFrame(art6_0_pdf_list, columns = ['pdf_file'])

df_60['id'] = df_60.index
df_60['batch'] = ((df_60['id'])/50).astype(int)

In [7]:
df_60.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4908 entries, 0 to 4907
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pdf_file  4908 non-null   object
 1   id        4908 non-null   int64 
 2   batch     4908 non-null   int32 
dtypes: int32(1), int64(1), object(1)
memory usage: 96.0+ KB


In [8]:
df_60['batch'].value_counts()

0     50
74    50
72    50
71    50
70    50
      ..
30    50
29    50
28    50
27    50
98     8
Name: batch, Length: 99, dtype: int64

In [10]:
len(df_60['batch'].value_counts())

99

In [9]:
# parse pdf
for i in range(0,100): #0-99
    globals()[f'df_6.0_{i}'] = clean_parsed_pdf_section(parse_pdf_section(tqdm(df_60[df_60['batch']==i]['pdf_file'])))
    print("Batch {} completed".format(i))

    # save json file name
    date = datetime.date.today().strftime('%Y_%m_%d')

    file_name = f"../../../data/parsed/parsed_art6.0_batch{i}_{date}.json"
    if os.path.exists(file_name):
        os.remove(file_name)

    # save file as json
    globals()[f'df_6.0_{i}'].to_json(file_name)

100%|██████████| 50/50 [00:44<00:00,  1.11it/s]


Batch 1 completed


100%|██████████| 50/50 [00:51<00:00,  1.02s/it]


Batch 2 completed


100%|██████████| 50/50 [00:31<00:00,  1.57it/s]


...

Batch 97 completed


100%|██████████| 8/8 [00:05<00:00,  1.51it/s]


Batch 98 completed


0it [00:00, ?it/s]
0it [00:00, ?it/s]


KeyError: 'simp_text'

#### Article 9.3 (129 cases)

In [11]:
# create dataframe
df_93 = pd.DataFrame(art9_3_pdf_list, columns = ['pdf_file'])

df_93['id'] = df_93.index
df_93['batch'] = ((df_93['id'])/100).astype(int)

In [12]:
df_93['batch'].value_counts()

0    100
1     92
Name: batch, dtype: int64

In [15]:
# parse pdf
for i in range(0,1): #0-1
    globals()[f'df_9.3_{i}'] = clean_parsed_pdf_section(parse_pdf_section(tqdm(df_93[df_93['batch']==i]['pdf_file'])))
    print("Batch {} completed".format(i))

    # save json file name
    date = datetime.date.today().strftime('%Y_%m_%d')

    file_name = f"../../../data/parsed/parsed_art9.3_batch{i}_{date}.json"
    if os.path.exists(file_name):
        os.remove(file_name)

    # save file as json
    globals()[f'df_9.3_{i}'].to_json(file_name)

100%|██████████| 100/100 [04:53<00:00,  2.93s/it]


Batch 0 completed


In [13]:
# parse pdf
for i in range(1,2): #0-1
    globals()[f'df_9.3_{i}'] = clean_parsed_pdf_section(parse_pdf_section(tqdm(df_93[df_93['batch']==i]['pdf_file'])))
    print("Batch {} completed".format(i))

    # save json file name
    date = datetime.date.today().strftime('%Y_%m_%d')

    file_name = f"../../../data/parsed/parsed_art9.3_batch{i}_{date}.json"
    if os.path.exists(file_name):
        os.remove(file_name)

    # save file as json
    globals()[f'df_9.3_{i}'].to_json(file_name)

100%|██████████| 92/92 [09:26<00:00,  6.15s/it]


Batch 1 completed


# Append all dataframes

In [42]:
import glob
import os 

path = f"../../../data/parsed/*.json"
files = glob.glob(path)

In [190]:
data_merged = pd.concat([pd.read_json(f) for f in files])

In [191]:
data_merged=data_merged.reset_index(drop=True)

In [192]:
# change section_text to simp_text where len_pdf <= 5 or section == "None"
data_merged.loc[data_merged["len_pdf"] <= 5, "section_text"] = data_merged.loc[data_merged["len_pdf"] <= 5, "simp_text"]

In [193]:
import re

def _parse_simplified_text2(text):
    # Define the start and end patterns
    start_pattern = re.compile(r'(?i)business activities of the undertakings concerned are|business activities')
    end_pattern = re.compile(r'(?i)after examination of the notification|\n \n|\n\n')

    result = "None"
    try:
        # Find the start and end positions of the patterns
        start_pos = start_pattern.search(text).end()
        end_match = end_pattern.search(text[start_pos:])
        if end_match is None:
            end_pos = len(text)
        else:
            end_pos = start_pos + end_match.start()

        # Extract the text between the patterns
        result = text[start_pos:end_pos].strip()
    except AttributeError:
        pass
    except UnboundLocalError:
        pass

    return result


In [194]:
data_merged.loc[data_merged['section_text'] == "None", "section_text"] = data_merged.loc[data_merged['section_text'] == "None", "bsn_act"].apply(_parse_simplified_text2)

In [195]:
len(data_merged[data_merged['section_text'] == "None"])

47

In [196]:
# # Join the texts in 'section_text' for each row using '\n' as the separator
data_merged['sec_text'] = data_merged['section_text'].apply(lambda lst: '\n'.join(lst))

# Reset the index of the DataFrame
data_merged = data_merged.reset_index(drop=True)

In [197]:
len(data_merged[data_merged['sec_text'] == ""])

1

In [198]:
# Drop if section_text is "None"
data_merged = data_merged[data_merged['section_text'] != "None"].reset_index(drop=True)

In [199]:
# Drop if sec_text is ""
data_merged = data_merged[data_merged['sec_text'] != ""].reset_index(drop=True)

In [201]:
data_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28997 entries, 0 to 28996
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   date          28997 non-null  object 
 1   year          26085 non-null  float64
 2   len_pdf       28997 non-null  int64  
 3   article       28997 non-null  object 
 4   article_txt   28997 non-null  object 
 5   article_62    28997 non-null  object 
 6   case_num      28997 non-null  object 
 7   filename      28997 non-null  object 
 8   section_text  28997 non-null  object 
 9   bsn_act       28997 non-null  object 
 10  simp_text     28997 non-null  object 
 11  section       25587 non-null  object 
 12  sec_text      28997 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 2.9+ MB


In [202]:
data_merged['sec_text']

0        EUROPEAN COMMISSION\nBrussels, 05.03.2021\nC(2...
1        In the published version of this decision,\nso...
2        Dear Sir or Madam,\n(1)  Following a referral ...
3        (2)  Microsoft is a global technology company,...
4        (4)  The Transaction will be implemented by me...
                               ...                        
28992    :\n \n\n\n\n \n \nf\no\nr\n \n \nC\no\nr\nd\n...
28993    :\n \n\n\n\n \n \nf\no\nr\n \nO\nn\ne\nx\n:\n...
28994    :\n \n\n\n\n \n \nR\nE\nW\nE\n \ni\ns\n \nm\n...
28995    :\n \n\n\n–\n \nf\no\nr\n \nC\na\nr\ng\ni\nl\n...
28996    :\n \n\n\n\n \n \nf\no\nr\n \nA\nM\nC\n:\n \n...
Name: sec_text, Length: 28997, dtype: object

In [203]:
# save json file name
date = datetime.date.today().strftime('%Y_%m_%d')

file_name = f"../../../data/processed/data_merged_{date}.json"
if os.path.exists(file_name):
    os.remove(file_name)

# save file as json
data_merged.to_json(file_name)